# Geting tv listings from popular brands

In [1]:
import requests
import csv

# eBay API Credentials (Use Production Key)
APP_ID = "RajaRedd-Test-PRD-9ded19238-abc5a091"  

# eBay Finding API Endpoint (Production)
url = "https://svcs.ebay.com/services/search/FindingService/v1"

# List of TV Brands and Sizes to Search
tv_brands = ["Samsung", "LG", "Sony", "TCL", "Hisense", "Vizio", "Philips", "Panasonic"]
tv_sizes = ["43 Inch", "50 Inch", "55 Inch", "65 Inch", "75 Inch", "85 Inch"]

# Create a CSV file to store results
csv_filename = "ebay_tv_listings.csv"

# Open CSV File
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:

    writer = csv.writer(file)
    writer.writerow(["Brand", "Size", "Item ID", "Model", "Price", "Currency"])

    # Loop through all brands and sizes
    for brand in tv_brands:
        for size in tv_sizes:
            search_query = f"{brand} {size} TV"
            print(f"🔍 Searching for: {search_query}")

            # API Parameters
            params = {
                "OPERATION-NAME": "findItemsByKeywords",
                "SERVICE-VERSION": "1.0.0",
                "SECURITY-APPNAME": APP_ID,
                "RESPONSE-DATA-FORMAT": "JSON",
                "REST-PAYLOAD": "",
                "keywords": search_query,
                "paginationInput.entriesPerPage": 5,  # Fetch top 5 listings per query
            }

            # Make API Request
            response = requests.get(url, params=params)
            data = response.json()

            # Extract TV Listings
            try:
                for item in data["findItemsByKeywordsResponse"][0]["searchResult"][0]["item"]:
                    item_id = item["itemId"][0]
                    title = item["title"][0]
                    price = item["sellingStatus"][0]["currentPrice"][0]["__value__"]
                    currency = item["sellingStatus"][0]["currentPrice"][0]["@currencyId"]

                    # Save to CSV File
                    writer.writerow([brand, size, item_id, title, float(price), currency])

                    print(f"✅ Found: {title} - {price} {currency}")

            except KeyError:
                print(f"❌ No listings found for {search_query}")

print(f"\n📁 TV Listings saved to {csv_filename}")


🔍 Searching for: Samsung 43 Inch TV
✅ Found: Samsung 43" TU690T LED 4K UHD Smart Tizen TV UN43TU690TFXZA - 239.0 USD
✅ Found: Samsung QN43Q60D 43 Inch QLED 4K Smart TV (2024) - Open Box - 355.0 USD
✅ Found: SAMSUNG 43" Class 4K Ultra HD (2160P) HDR Smart QLED TV QN43Q60T - 200.0 USD
✅ Found: SAMSUNG 43" Class 4K UHD (2160p) Tizen Smart LED TV (UN43DU7200B) - 239.99 USD
✅ Found: Samsung QN43LS03DA 43-Inch The Frame QLED 4K Smart TV (2024) - Open Box - 649.99 USD
🔍 Searching for: Samsung 50 Inch TV
✅ Found: Samsung - 50” Class DU6900 Series Crystal UHD 4K Smart Tizen TV UN50DU6900 - 235.0 USD
✅ Found: Samsung AU8000 50 inch 2160p Crystal UHD LED Smart TV - 150.0 USD
✅ Found: Samsung 50" Class Crystal UHD 4K (2160p) LED Smart TV UN50DU6900F-NEW SESAL OPEN - 298.0 USD
✅ Found: Samsung 50" DU7200 Class Crystal 4K UHD LED  Smart TV - 387.9 USD
✅ Found: Samsung QN50LS03DAF 50 inch The Frame QLED 4K Smart TV QN50LS03DAFXZA (2024) - 810.0 USD
🔍 Searching for: Samsung 55 Inch TV
✅ Found: samsung

# tracking prices using item id for every 6 hrs

In [9]:
!pip install schedule

In [11]:
import requests
import sqlite3
import csv
import time
from datetime import datetime
import schedule

# eBay API Credentials
ACCESS_TOKEN = "v^1.1#i^1#f^0#r^0#I^3#p^3#t^H4sIAAAAAAAAAOVZf2wbVx2Pk7QlrClojBZKR7Nrp0mrzn5357PvjtiVF7tN2jh2bCdtAlt4d/fOfs757nr3LokjIdL+MUQl2FDHEJoGhQ7xRzUQEhJiEy0MbRoj0hjbpAlVk+gQA9EBG4MM0Dbu7CR1A20Tu9oscf8k9+776/N93x/+vgcWNvfcee/gvUu9gS2dpxfAQmcgwNwEejZv2retq3Pnpg7QQBA4vbB3oftE1x/6HVjRLSmHHMs0HNQ3V9ENR6otxijXNiQTOtiRDFhBjkQUKZ9ID0tsEEiWbRJTMXWqbygZo+SwBmUeAk1WGFHjIt6qsSKzYMYoIRJlAWIhx6t8mEHeZ8dx0ZDhEGiQGMUClqcBSwOhwHISG5VANBiNCpNU3ziyHWwaHkkQUPGatVKN124w9dqWQsdBNvGEUPGhxIF8JjGUTI0U+kMNsuLLbsgTSFznyrcBU0V941B30bXVODVqKe8qCnIcKhSva7hSqJRYMaYJ82ueFkGY5yJhDQhcGHJy5Ia48oBpVyC5th3+ClZprUYqIYNgUr2eRz1vyGWkkOW3EU/EULLP/zPqQh1rGNkxKnVXYmIsn8pRffls1jZnsIpUHykbZSJiNCKKIhW3YRmqpgGWddQFLXt4jZIB01Cx7y+nb8QkdyHPYLTWLUyDWzyijJGxExrxjVmlEwuAWXFfRJz097O+gS4pGf6Woorng77a6/WdvxINl/f/RsUDxwNW5BjAsHyEi4jgKvHg5/qGYiLub0simw35tiAZVukKtKcRsXSoIFrx3OtWkI1VieM1lhM0RKsRUaPDoqbRMq9GaEZDCCAky4oo/J+EBiE2ll2CVsNj7YcavhiVV0wLZU0dK1VqLUmt0iwHw5wTo0qEWFIoNDs7G5zlgqZdDLEAMKGj6eG8UkIVSK3S4usT07gWFopXgD16iVQtz5o5L+o85UaRinO2moU2qeaRrnsLKzF7hW3xtatXATmgY88DBU9Fe2EcNB2C1JagqWgGK2gKq+2FjGU5EQBR8HOdAwBwLYHUzSI20oiUzDaDmUonhoZbguaVT0jaC1RDEWIiy0UoInI08P4DLYFNWNZQpeISKOtoqM22kmd4VuBbgme5brvloV6d5UvqvH0szLYEze+6EoaaRMxpZPzvSurn+vuJNZc6kEvlB6cKmcOpkZbQ5pBmI6dU8LG2W5wmRhOHE96TzhyyB8vqhF49JOMq5xYBzxdzCXNsYl6R89nBI6lJO5s5aB0aSab0gWL5QDo5EB5wJ45GZ9D4vjFjIhGLteSkPFJs1GalKxMOjUJroJg/6I4dFNmcAlNH0wWBreij2Wph38AoM+zO65rDKEOtgU8X2y3TVzpu6922cPUUXwXo5/r7ANKuJ+ZUrQpNeW8tAU0V265eq4rACZBVGDEMYATxUIxqQFY5zXsgj8SW22+b4c15E1MOqSpdQA6hs7kkLXpzFSN6YxwNZYWHQGRa7MnttsU3qiU7/uT2XkDzc3398HwZjicEWjjo/2oIKmYlZEKXlPylqZrVfeshCjne5BesT/qe5KCN/NlarzbDvAEebMx4s6JpV5tRuMq8AR6oKKZrkGbULbNugENzdQ3run8g0IzCBvaNmGlAvUqw4jSlEht+tDkbYLFgtQZQxY7l58u6OL21CrIVFMRq/UCxGWNt5CmEtTO0Zpg2qHLVZMMkWMNKXYbjyo5iY2v9VjTI8XP9KrKa8Yfj5cKGtq7OsC5VDVxIRTqeQXa1tVEcqdhGCplybdxeLcPvklN+m6xOJU1Dpdd0TVopHdOIM2+1BN93ajuesWQT+fyRTC7ZErgkmmm3nz6AQxzDqJAOa2GWDgsRgRYYAdCKEOWiYRlpYri144jrHSx1H3/5PQfNRL0phY+ybGS90NYsNJxn/9ctRujKC8R4R+1hTgSeACcC5zoDAdAPbmf2gNs2d411d23d6WDilXuoBR1cNCBxbRScRlULYrvzIx3PbhtWjw8O/31Bdn905M39Qkdvw/3l6bvBx1ZvMHu6mJsarjPBrstfNjEf2tHL8oAFAsuxURCdBHsuf+1mtnffMlg+d/o+6/jsrY89/fzFR97avvjow8+C3lWiQGBTR/eJQMeDc3/ddfN9T7k/39+/eGZw75+/mv/w987/8fbCzRFc3vXRJ/9kfuIrl+554/5TYPiWX9x6dvaZ4ZPUL3+r/O3k4nPx7/yufCrw2f60kHncLm6//7Yll936xvTYFx5jH3nyn+985sXYW7sfqo6I1V/Zu/9CndyzSMZ2fJ45tePk2+XX5xa++8rOS730M7+/8I8X8KufvOPwlz/98uP/fuLs19EXo8c+93Z5/JvvfP+l0u4fbzn6s28kPzh9z5mnFvbTP3ltnE/kX3n13Qe27n1p+k3l0uIPv/3xn/Y8cHH+a8fyF+kLH3hwaeTuLS/c0fPc9ul9kRfPfWrb7oPzMlo6/7y65dEfTJ4vwl+/e/Zb//pScOnMhadfm5PuxK//pr6X/wGEeZZtWR4AAA=="  # Replace with your eBay OAuth Token
EBAY_API_URL = "https://api.ebay.com/buy/browse/v1/item/{}"


# Load TV Item IDs from CSV
csv_filename = "ebay_tv_listings.csv"

def load_tv_item_ids():
    """ Load TV item IDs from CSV file """
    tv_items = []
    try:
        with open(csv_filename, mode="r", encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                tv_items.append((row[2], row[1], row[3]))  # Item ID, Size, Model
    except FileNotFoundError:
        print(f"❌ Error: {csv_filename} not found. Ensure you have fetched TV item IDs first.")
    return tv_items

# Initialize SQLite Database
def setup_database():
    """ Create database and table if not exists """
    conn = sqlite3.connect("ebay_tv_prices.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS tv_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            time TEXT,
            item_id TEXT,
            model TEXT,
            size TEXT,
            price REAL,
            currency TEXT
        )
    """)
    conn.commit()
    conn.close()

# Fetch and Store Prices
def track_prices():
    """ Fetch and store TV prices from eBay """
    print(f"\n🔄 Running price tracker at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    tv_items = load_tv_item_ids()
    if not tv_items:
        print("❌ No valid Item IDs found. Exiting tracker.")
        return

    conn = sqlite3.connect("ebay_tv_prices.db")
    cursor = conn.cursor()

    try:
        for item_id, size, model in tv_items:
            print(f"🔍 Checking Price for {model} {size} (Item ID: {item_id})")

            headers = {
                "Authorization": f"Bearer {ACCESS_TOKEN}",
                "Content-Type": "application/json"
            }

            response = requests.get(EBAY_API_URL.format(item_id), headers=headers)

            if response.status_code == 200:
                data = response.json()
                price = data["price"]["value"]
                currency = data["price"]["currency"]

                current_date = datetime.now().strftime("%Y-%m-%d")
                current_time = datetime.now().strftime("%H:%M:%S")

                cursor.execute("""
                    INSERT INTO tv_prices (date, time, item_id, model, size, price, currency)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                """, (current_date, current_time, item_id, model, size, float(price), currency))
                conn.commit()

                print(f"✅ Price Recorded: {model} {size} - {price} {currency} at {current_time}")

            elif response.status_code == 404:
                print(f"⚠️ Item ID {item_id} not found! Skipping...")
                continue  # Skip to the next item

            elif response.status_code == 401:
                print("🔒 OAuth Token Expired! Please refresh the token and restart the script.")
                return  # Exit script if token expires

            else:
                print(f"❌ API Request Failed! Status Code: {response.status_code}")
                print(response.json())  # Print full error response

    except Exception as e:
        print(f"⚠️ Unexpected error occurred: {e}")

    finally:
        conn.close()
        print("\n📊 Price tracking completed. Next run in 6 hours.")


In [13]:
track_prices()



🔄 Running price tracker at 2025-02-11 02:25:17
🔍 Checking Price for Samsung 43" TU690T LED 4K UHD Smart Tizen TV UN43TU690TFXZA 43 Inch (Item ID: 235718474991)
🔒 OAuth Token Expired! Please refresh the token and restart the script.

📊 Price tracking completed. Next run in 6 hours.
